In [182]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Pré-processamento #

### Base e encoder manual ###

In [183]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [184]:
warnings.filterwarnings("ignore")

In [185]:
df_encod_manual = pd.DataFrame.copy(df)

In [186]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [187]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [188]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [189]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [190]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [191]:
previsores_label = df.iloc[:, 0:11].values

In [192]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [193]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [194]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [195]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [196]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [197]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [198]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

# Previsores #
<span style="font-size: 13px;">
<li> <strong>previsores</strong> -> Atributos codificados manualmente sem escalonamento.</li></br>
<li> <strong>previsoresHot_esc</strong> -> Atributos codificados com LabelEncoder e OneHotEncoder e escalonados.</li></br>
<li> <strong>previsores_esc</strong> -> Atributos codificados manualmente e escalonados.</li></br>
<li> <strong>previsores_hot</strong> -> Atributos codificados com OneHotEncoder sem escalonamento.</li></br>
<li> <strong>previsores_label</strong> -> Atributos codificados com LabelEncoder e sem escalonamento. </li></br>
</span>

# Árvores de Decisão #

<span style="font-size: 12.5px;">

**Introdução às Árvores de Decisão**

As árvores de decisão são modelos de aprendizado de máquina que representam uma sequência de decisões em forma de uma estrutura de árvore. Elas são usadas para tarefas de classificação e regressão, onde o objetivo é prever a classe de uma observação (classificação) ou o valor de uma variável de destino (regressão) com base em várias características ou atributos.

Funcionamento Básico

O funcionamento das árvores de decisão pode ser resumido em alguns passos principais:

1. **Divisão dos Dados**: A árvore começa com todos os dados no nó raiz e seleciona a melhor característica para dividir os dados em subconjuntos mais homogêneos.

2. **Escolha da Melhor Divisão**: A melhor característica para divisão é escolhida com base em critérios como o índice Gini (para classificação) ou a redução da variância (para regressão).

3. **Construção da Árvore**: A árvore é construída recursivamente, dividindo os dados em subconjuntos menores até que uma condição de parada seja alcançada.

4. **Poda (Pruning)**: Após a construção da árvore, é comum podar partes da árvore que não contribuem significativamente para a sua capacidade de generalização.

**Conceitos Importantes**

Alguns conceitos importantes relacionados às árvores de decisão incluem:

- **Nós (Nodes)**: Representam pontos de divisão na árvore, onde uma decisão é tomada com base em uma característica.
- **Ramos (Branches)**: Representam os caminhos que seguem a partir de um nó para os nós filhos.
- **Folhas (Leaves)**: Representam as saídas finais da árvore, onde uma decisão é tomada.

</span>
</br>
<span style="font-size: 12.5px;">


**Índice Gini (para classificação):**

$ 
Gini(D) = 1 - \sum_{i=1}^{k} (p_i)^2 
$

onde $D$ é o conjunto de dados e $k$ é o número de classes. $p_i$ é a proporção de amostras da classe $i$ em $D$.


**Entropia (para classificação):**

A entropia é uma medida de impureza de um conjunto de dados. Quanto maior a entropia, maior a incerteza sobre a classe das amostras no conjunto de dados. A fórmula para calcular a entropia é dada por:

$
\text{Entropia}(D) = - \sum_{i=1}^{k} p_i \log_2(p_i)
$

onde $D$ é o conjunto de dados, $k$ é o número de classes, e $p_i$ é a proporção de amostras da classe $i$ em $D$.

**Ganho de Informação:**

O ganho de informação é uma medida da quantidade de redução da entropia que resulta da divisão de um conjunto de dados com base em uma determinada característica. É usado para selecionar a melhor característica para divisão em árvores de decisão. O ganho de informação é calculado como a diferença entre a entropia do conjunto de dados original e a entropia ponderada dos subconjuntos resultantes da divisão.

$
\text{Gain}(D, \text{feature}) = \text{Entropia}(D) - \sum_{j} \frac{|D_j|}{|D|} \text{Entropia}(D_j)
$

onde $D$ é o conjunto de dados original, $D_j$ é o subconjunto resultante da divisão, e $\text{feature}$ é a característica usada para divisão.

**Comparação com o Índice Gini:**

Tanto o índice Gini quanto a entropia são critérios comuns para medir a qualidade da divisão em árvores de decisão. Enquanto o índice Gini é calculado com base nas frequências das classes, a entropia é calculada com base na proporção das classes. Ambos os critérios são amplamente utilizados e geralmente produzem resultados semelhantes.


**Parâmetros Importantes**

Alguns parâmetros importantes em árvores de decisão incluem:

- **Critério**: Define a função para medir a qualidade da divisão (por exemplo, "gini" ou "entropy" para classificação, "mse" para regressão).
- **Profundidade Máxima**: Limita a profundidade da árvore para evitar overfitting.
- **Número Mínimo de Amostras em Folha**: Define o número mínimo de amostras necessário em uma folha.
- **Número Mínimo de Amostras para Divisão**: Define o número mínimo de amostras necessário para realizar uma divisão em um nó.

</span>


In [229]:
arvore = DecisionTreeClassifier(
    criterion = 'gini',
    random_state = 5,
    max_depth = 4
)

In [230]:
arvore.fit(x_train, y_train);

## Previsoes ##

**Dados de Teste**

In [231]:
previsoes_test = arvore.predict(x_test)

**Dados de Treino**

In [232]:
previsoes_train = arvore.predict(x_train)

## Métricas ##

**Dados de Teste**

In [233]:
acuracia_test = accuracy_score(y_test, previsoes_test) * 100.0
print("Acurácia: %.2f%%" % (acuracia_test))

Acurácia: 81.16%


In [234]:
print("Matriz de Confusão: \n", confusion_matrix(y_test, previsoes_test))

Matriz de Confusão: 
 [[ 91  30]
 [ 22 133]]


In [235]:
print("Relatório: \n", classification_report(y_test, previsoes_test))

Relatório: 
               precision    recall  f1-score   support

           0       0.81      0.75      0.78       121
           1       0.82      0.86      0.84       155

    accuracy                           0.81       276
   macro avg       0.81      0.81      0.81       276
weighted avg       0.81      0.81      0.81       276



**Dados de Treino**

In [236]:
acuracia_train = accuracy_score(y_train, previsoes_train) * 100.0
print("Acurácia: %.2f%%" % (acuracia_train))

Acurácia: 88.14%


In [237]:
print("Matriz de Confusão: \n", confusion_matrix(y_train, previsoes_train))

Matriz de Confusão: 
 [[239  50]
 [ 26 326]]


In [238]:
print("Relatório: \n", classification_report(y_train, previsoes_train))

Relatório: 
               precision    recall  f1-score   support

           0       0.90      0.83      0.86       289
           1       0.87      0.93      0.90       352

    accuracy                           0.88       641
   macro avg       0.88      0.88      0.88       641
weighted avg       0.88      0.88      0.88       641

